*please execute the cell below before starting the tutorial by selecting the cell and pressing Ctrl+Enter*

In [ ]:
from aiida.orm import Dict, load_node
from aiida.engine import submit

from aiida import load_profile

load_profile()

# Workchains and high-throughput study

In the final, sixth part of the AiiDA-FLEUR tutorial, we will cover the main part of AiiDA - workchains. Workchains are key-turn solutions for particular tasks such as finding equation of states or performing structure optimisation. The main goal of a workchain is to automatically perform a certain task keeping the history of all performed actions. The main advantage of using workchains is providing a low entering threshold for newcomers and routine tasks automatization for more advanced users.

In this part of the tutorial you will explore and learn how to work with AiiDA-Fleur workchains.

Workchains are similar to `CalcJob`: basically they can be treated as a black box using user's input and producing the output. For example, the SCF workchain is an engine that produces self-consistent charge density (and other interesting parameters) for a given structure. Equation Of States (EOS) workchain calculates equation of states for a given structure and etc.

<img src="files/images/black_box.png" width="1000">

The power of workchains is hidden behind their possible use in other workchains. Workchain A can be used in a workchain B, workchain C can use workchains A and B etc. Using smaller workchains as building blocks for a more complex algorithms, one can develop hierarchic structure of the task that AiiDA can perform.

## Workchain hierarchy 

The hierarchy of all implemented workchains in AiiDA-Fleur v1.0.0 is shown below. Black arrows mean the use of a workchain by another one. For instance, the geometry optimization workchain uses the SCF workchain inside it.
It is clearly seen that every higher-level workchain uses the SCF workchain.

<img src="files/images/workchains.png" width="800">


In next section we will cover the central workchain of the AiiDA-FLEUR plugin: the SCF workchain.

## SCF WorkChain 

The SCF workchain is responsible for converging charge density for a given structure. It submits a Fleur calculation several times until the convergence criterion or maximal number of Fleur submissions are reached.

### Inputs of the SCF WorkChain

|name|type|description| required |
|:---:|:---:|:---------:|:---:|
|fleur | Code | Fleur code | yes |
|inpgen | Code | Inpgen code| no |
|wf_parameters | Dict | Settings of the workchain| no |
|structure | StructureData | Structure data node| no |
|calc_parameters | Dict | FLAPW parameters, used by inpgen| no |
|fleurinp | FleurinpData | FLEUR input files| no |
|remote_data | RemoteData | Remote folder of another calculation| no |
|options | Dict | AiiDA options (computational resources)| no |
|settings | Dict | special settings for Fleur calculation| no |

Similarly to the `FleurCalculation`, SCF workchain has only one required input. Again, one *must* provide one of the [supported](https://aiida-fleur.readthedocs.io/en/v1.1.0/user_guide/workflows/scf_wc.html#layout) input configurations to ensure the predictable behaviour of the workchain:

* **fleur** + **fleurinp**
* **fleur** + **fleurinp** + **remote_data**
* **fleur** + **remote_data**
* **fleur** + **inpgen** + **structure**

In this tutorial we will cover **fleur** + **inpgen** + **structure** configuration only, which makes the workchain to submit a single inpgen calculation followed by several Fleur code submissions.

## SCF workchain submission

### Inputs preparation

We need import the `FleurScfWorkChain` first:

In [ ]:
from aiida_fleur.workflows.scf import FleurScfWorkChain

Next, let us setup input parameters for the workchain that control its behaviour. A cell below sets the maximal number of Fleur submission to 3, density convergence criterion to 0.001, maximal number of iterations in a single `FleurCalculation` to 30.

In [ ]:
wf_para = Dict(dict={'fleur_runmax' : 3,           # maximal number of Fleur submissions
                     'density_converge' : 0.001,   # density convergence criterion
                     'mode' : 'density',           # density is converged
                     'itmax_per_run' : 30,         # number of scf iterations in each Fleur submission
                     'serial' : False})            # use mpi submission

We are going to use StructureData node created in the tutorial number 1. Let us use the stored structure:

In [ ]:
# you need to modify this - remind the PK of the silicon structure SI_PK
structure = load_node(SI_PK)

Despite **options** and **wf_parameters** are never required, we will set them up because we do not want to use default values.

In [ ]:
options = Dict(dict={'resources' : {"num_machines": 1, "num_mpiprocs_per_machine" : 2},
                     'withmpi' : True,
                     'max_wallclock_seconds' : 600})

calc_parameters = Dict(dict={
    'kpt': {
        'div1': 2,
        'div2' : 2,
        'div3' : 2
        }})

**Note**: inpgen calculation submitted by SCF workchain is always submitted with an option:
 
    {'resources' : {"num_machines": 1, "num_mpiprocs_per_machine" : 1}, 'withmpi' : False}
    
which means the resources specified in SCF input apply to `FleurCalculation` only.

Finally, we need to load Fleur and inpgen nodes:

In [ ]:
fleur_code = load_node(146)
inpgen_code = load_node(7)

### Job submission

In contrast to `FluerinpgenCalculation` and `FleurCalculation`, there is no need to assemble inputs in a single dictionary (but one can do it for convenience).

In [ ]:
SCF_workchain = submit(FleurScfWorkChain,
                       fleur=fleur_code,
                       inpgen=inpgen_code,
                       calc_parameters=calc_parameters,
                       structure=structure,
                       wf_parameters=wf_para,
                       options=options)
print('Submitted SCF workchain pk={}'.format(SCF_workchain.pk))

Now we can check the status of the workchain simply executing a cell below:

In [ ]:
# you need to modify this
!verdi process status PK

You can execute the cell above again and again until it does not say the task is finished. The output for the finished workchain should look like:

<img src="files/images/scf_print.png" width="400">


You can also check all processes submitted last 24 hours by:

In [ ]:
!verdi process list -a -p 1

### Results analysis

SCF workchain returns three outputs:

|name| type | comment|
|:--:|:----:|:----------:|
|output_scf_wc_para|Dict| results of the workchain|
|fleurinp| FleurinpData| FleurinpData that was used (after all modifications) |
|last_fleur_calc_output| Dict | shortcut for last `FleurCalculation` output dict |

You can find all output node PKs running:

In [ ]:
# you need to modify this - replace SCF_PK
!verdi process show SCF_PK

Some data, given in `output_scf_wc_para` can be visualised via:

In [ ]:
# you need to modify this - replace SCF_PK
%matplotlib inline
from aiida_fleur.tools.plot.fleur import plot_fleur
plot_fleur(SCF_PK)

Finally, you can access output_scf_wc_para dictionary via:

In [ ]:
# you need to modify this - replace SCF_PK
scf_wc = load_node(SCF_PK)
scf_wc.outputs.output_scf_wc_para.get_dict()

or

In [ ]:
# you need to modify this - replace OUTPUT_DICT_PK
!verdi data dict show OUTPUT_DICT_PK

## Higher-level workchains

And now - finally - we will see the real power of the workchains. To get a general understanding how higher-level workchains work, we will use the Equation Of States (EOS)workchain.

Note, SCF workchain is submitted inside EOS workchain, thus we will expect inputs that control neted SCF workchain.

### Equation of States workchain

Please also refer to the [documentation](https://aiida-fleur.readthedocs.io/en/v1.1.0/user_guide/workflows/eos_wc.html).

#### Input preparation and submission
First of all, we need to import the workchain:

In [ ]:
from aiida_fleur.workflows.eos import FleurEosWorkChain

Table below lists all possible input nodes of EOS:

|name|type|description| required |
|:---:|:---:|:---------:|:---:|
|scf | namespace | inputs for nested SCF WorkChain | no |
|wf_parameters | Dict | Settings of the workchain| no |
|structure | StructureData | Structure data node| no |

scf is a namespace and contains all the inputs for nested SCF workchains (except `structure`, `fleurinp` and `remote_data`).

Since EOS does not explicitly submits any CalcJob, there is no `options` that control parallisation neither other inputs for FleurCalculation. All of them should be specified in the `scf` namespace.

Let us first initialise *direct* inputs of the EOS:

In [ ]:
wf_para = Dict(dict={'points': 21,      # number of rescaled structures to calculate
                     'step': 0.002,     # volume ratio step between structures
                     'guess': 1.03      # cental volume ratio, i.e. 10 structures will have smaller valume
                    })                  # and 10 of the will have larger volume than 1.03

In [ ]:
structure = load_node(<structure_pk>)

And now we are going to prepare inputs for the nested `SCF` workchain. Note, we **must** use `inpgen` + `calc_parameters` + `structure` input pattern for `SCF` workchain, except this time **we do not need to pass** `structure` to scf namespace because it will be done inside `EOS` workchain automatically. The reason for this is that a single EOS workchain submits several SCF ones for different volumes, thus should leave `structure` input empty.

In [ ]:
options_scf = Dict(dict={'resources' : {"num_machines": 1, "num_mpiprocs_per_machine" : 2},
                         'withmpi' : True,
                         'max_wallclock_seconds' : 600})

wf_para_scf = Dict(dict={'fleur_runmax': 3,
                        'itmax_per_run': 60,
                        'density_converged': 0.02,
                        'serial': False,
                        'mode': 'density'
               })

calc_parameters_scf = Dict(dict={
    'kpt': {
        'div1': 4,
        'div2' : 4,
        'div3' : 4
        }})

fleur_code = load_node(???)
inpgen_code = load_node(???)

Finally, we can assemble the inputs and submit the workchain:

In [ ]:
inputs = {'scf': {
                  'wf_parameters' : wf_para_scf,
                  'calc_parameters' : calc_parameters_scf,
                  'options' : options_scf,
                  'inpgen' : inpgen_inp,
                  'fleur' : fleur_inp
                 },
          'wf_parameters' : wf_para,
          'structure' : structure
}

EOS_workchain = submit(FleurEosWorkChain, **inputs)
print('Submitted EOS workchain pk={} for {} structure'.format(EOS_workchain.pk, structure.get_formula()))

#### Result analysis

Similarly to FleurCalculation, one can analyse outputs of the EOS workchain printing out the general information of the EOS workchain node. There are two output nodes: the first one, `output_eos_wc_para`, contains some information about the workchain such as parameters of Birch-Murnaghan fit and others. The `output_eos_wc_structure` is a strucuture having the lowest energy according to Birch-Murnaghan equation.